# Pre-requisites

1. Install [.NET 5](https://dotnet.microsoft.com/download/dotnet/5.0)
2. Install [.NET Interactive Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension
3. Install [.NET Install Tool for Extension Authors](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension
4. Install [Data Table](https://marketplace.visualstudio.com/items?itemName=RandomFractalsInc.vscode-data-table) extension
5. Login into your Azure account on [Azure Account](https://marketplace.visualstudio.com/items?itemName=ms-vscode.azure-account) extension
6. Confirm that the Azure account you use to login can access your Power Apps environment

# Dataverse Interactive Notebook

This notebook demonstrates how you can use interactive notebooks with PowerPlatform .net core assembly

In [ ]:
#r "nuget:Microsoft.PowerPlatform.Dataverse.Client,*-*"
#r "nuget:Azure.Identity,*-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:Microsoft.Data.Analysis,*-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget: System.Runtime.Caching,*-*"

using Microsoft.DotNet.Interactive.Formatting;
using static System.Console;
using System.Linq;
using System.Collections;
using System.Collections.Generic;
using Azure.Core;
using Azure.Identity;
using Microsoft.PowerPlatform.Dataverse.Client;
using Microsoft.Xrm.Sdk;
using Microsoft.Xrm.Sdk.Metadata;
using Microsoft.Crm.Sdk.Messages;
using Microsoft.Data.Analysis;
using System.Text.Json;
using System.Runtime.Caching;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.ML;

The first step is to get the environment URL from the user. Visual Studio Code will prompt you for an environment URL when you run the next block. Provide an environment URL. Once the URL is provided, Azure Identity SDK can be used to get the token based on the current logged in user in VSCode. This user needs to have access to both Azure and Power Apps environment. DataverseClient will use Azure Identity SDK to retrieve the token. The client is now authenticated.

In [ ]:
var environment = await GetInputAsync("Enter Power Apps Environment URL. e.g. https://org.crm.dynamics.com");
ObjectCache cache = MemoryCache.Default;

var managedIdentity = new DefaultAzureCredential();
var client = new ServiceClient(
    tokenProviderFunction: f => GetToken(environment, managedIdentity, cache),
    instanceUrl: new Uri(environment));


private async Task<string> GetToken(string environment, DefaultAzureCredential credential, ObjectCache cache)
{
    //TokenProviderFunction is called multiple times, so we need to check if we already have a token in the cache
	var accessToken = cache.Get(environment);
	if (accessToken == null)
	{
		accessToken = (await credential.GetTokenAsync(new TokenRequestContext(new[] { $"{environment}/.default" })));
		cache.Set(environment, accessToken, new CacheItemPolicy { SlidingExpiration = TimeSpan.FromMinutes(50) });
	}
	else
	{
		WriteLine($"Getting token from cache: Expires: {((AccessToken)accessToken).ExpiresOn.ToLocalTime().ToString("s")}, {((AccessToken)accessToken).Token}");
	}
	return ((AccessToken)accessToken).Token;
}

Since the `ServiceClient` is now authenticated, we can easily use this for all Dataverse calls.

In [ ]:
$"Tenant: {client.OrganizationDetail.TenantId}\nOrganizationId: {client.OrganizationDetail.OrganizationId}\nFriendlyName: {client.OrganizationDetail.FriendlyName}"

Now let us try to get some metadata.

In [ ]:
var entities = client.GetEntityMetadata("systemuser", EntityFilters.Attributes);
var attributes = entities.Attributes.Select(x=>new {x.ColumnNumber, x.LogicalName, DisplayName = x.DisplayName.UserLocalizedLabel?.Label ?? ""}).OrderBy(x=>x.ColumnNumber);
JsonSerializer.Serialize(attributes) //We need to serialise so that it can be rendered by Data Table extension as a scrollable table. It will use the default formatter otherwise, which does not scroll.

[{"ColumnNumber":1,"LogicalName":"systemuserid","DisplayName":"User"},{"ColumnNumber":3,"LogicalName":"territoryid","DisplayName":"Territory"},{"ColumnNumber":4,"LogicalName":"organizationid","DisplayName":"Organization "},{"ColumnNumber":5,"LogicalName":"businessunitid","DisplayName":"Business Unit"},{"ColumnNumber":6,"LogicalName":"parentsystemuserid","DisplayName":"Manager"},{"ColumnNumber":7,"LogicalName":"firstname","DisplayName":"First Name"},{"ColumnNumber":8,"LogicalName":"salutation","DisplayName":"Salutation"},{"ColumnNumber":9,"LogicalName":"middlename","DisplayName":"Middle Name"},{"ColumnNumber":10,"LogicalName":"lastname","DisplayName":"Last Name"},{"ColumnNumber":11,"LogicalName":"personalemailaddress","DisplayName":"Email 2"},{"ColumnNumber":12,"LogicalName":"fullname","DisplayName":"Full Name"},{"ColumnNumber":13,"LogicalName":"nickname","DisplayName":"Nickname"},{"ColumnNumber":14,"LogicalName":"title","DisplayName":"Title"},{"ColumnNumber":15,"LogicalName":"internale

Now let us try some messages. For example, `RetrieveTotalRecordCountRequest` which gives us the table count (snapshot).

In [ ]:
var recordCountResponse = ((RetrieveTotalRecordCountResponse)client.Execute(new RetrieveTotalRecordCountRequest
{
    EntityNames = new string[] { "account", "contact", "systemuser" }
})).EntityRecordCountCollection;
JsonSerializer.Serialize(recordCountResponse) //We need to serialise so that it can be rendered by Data Table extension as a scrollable table. It will use the default formatter otherwise, which does not scroll.

[{"Key":"account","Value":107},{"Key":"contact","Value":63},{"Key":"systemuser","Value":93}]

[PowerPlatform-DataverseServiceClient](https://github.com/microsoft/PowerPlatform-DataverseServiceClient) is still in preview. So, don't use it in production yet. But, it is great to use for data wrangling and some quick fixes to your Dataverse environment. I use it in my [Dataverse LINQPad Driver](https://github.com/rajyraman/Dataverse-LINQPad-Driver) as well.

Happy building.

[Natraj](https://twitter.com/rajyraman)